# Combining & organizing data

## Libraries and settings

In [1]:
# Libraries
import os
import pandas as pd
import numpy as np
import fnmatch

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Importing the prepared rental apartments data

In [2]:
# Get current working directory
print(os.getcwd())

# Read data to pandas data frame
df_orig = pd.read_csv('apartments_data_prepared.csv', 
                      sep=',', 
                      encoding='utf-8')

# Copy of data with selected colums
df = df_orig[['web-scraper-order', 
              'address_raw', 
              'datetime', 'rooms', 
              'area', 
              'luxurious',
              'price',
              'price_per_m2']]

# Get number of rows and columns
print(df.shape)

# Show first records
df.head(5)

C:\Workspacezhaw\data_analytics\Woche 3\Materials for exercises-20221005
(914, 8)


,web-scraper-order,address_raw,datetime,rooms,area,luxurious,price,price_per_m2
0,1662023695-433,"Sunnenbergstrasse 15, 8633 Wolfhausen, ZH",2023-01-21 17:05:04,3.5,122,1,3180,26.065574
1,1662023745-820,"Lavaterstr. 63, 8002 Zürich, ZH",2023-01-21 17:05:04,2.5,78,0,3760,48.205128
2,1662023742-807,"Langfurrenstrasse 5c, 8623 Wetzikon ZH, ZH",2023-01-21 17:05:04,5.5,115,0,2860,24.869565
3,1662023804-1290,"Sandbuckweg 5A, 8157 Dielsdorf, ZH",2023-01-21 17:05:04,3.5,74,0,2165,29.256757
4,1662023739-771,"Parkring 59, 8002 Zürich, ZH",2023-01-21 17:05:04,5.5,195,1,6900,35.384615


## Combining data from different sources

### Reading rental apartment data with geocoded addresses

In [3]:
# Meaning of variables
# lat: geographical latitude
# lon: geographical longitude
# bfs_number: official municipality id
# bfs_name: official municipality name

# Geocoded data (i.e. data with latitude and longitude)
df_geo = pd.read_csv('apartments_data_geocoded.csv', 
                     sep=',', 
                     encoding='utf-8')
df_geo.head(5)

,web-scraper-order,address_raw,lat,lon,bfs_number,bfs_name
0,1662023695-433,"Sunnenbergstrasse 15, 8633 Wolfhausen, ZH",47.255714,8.804976,112,Bubikon
1,1662023720-634,"Blumenbergstrasse 7, 8633 Wolfhausen, ZH",47.254879,8.793746,112,Bubikon
2,1662023745-834,"8608 Bubikon, ZH",47.277386,8.800306,112,Bubikon
3,1662023701-503,"8608 Bubikon, ZH",47.277386,8.800306,112,Bubikon
4,1662023745-820,"Lavaterstr. 63, 8002 Zürich, ZH",47.361378,8.533339,261,Zürich


### Join geo-information to rental apartment data using .merge()

In [4]:
df2 = df.merge(df_geo[['web-scraper-order', 
                       'lat', 
                       'lon', 
                       'bfs_number', 
                       'bfs_name']], 
               on="web-scraper-order")
df2.head()

,web-scraper-order,address_raw,datetime,rooms,area,luxurious,price,price_per_m2,lat,lon,bfs_number,bfs_name
0,1662023695-433,"Sunnenbergstrasse 15, 8633 Wolfhausen, ZH",2023-01-21 17:05:04,3.5,122,1,3180,26.065574,47.255714,8.804976,112,Bubikon
1,1662023745-820,"Lavaterstr. 63, 8002 Zürich, ZH",2023-01-21 17:05:04,2.5,78,0,3760,48.205128,47.361378,8.533339,261,Zürich
2,1662023742-807,"Langfurrenstrasse 5c, 8623 Wetzikon ZH, ZH",2023-01-21 17:05:04,5.5,115,0,2860,24.869565,47.328632,8.810400,121,Wetzikon (ZH)
3,1662023804-1290,"Sandbuckweg 5A, 8157 Dielsdorf, ZH",2023-01-21 17:05:04,3.5,74,0,2165,29.256757,47.477493,8.456285,86,Dielsdorf
4,1662023739-771,"Parkring 59, 8002 Zürich, ZH",2023-01-21 17:05:04,5.5,195,1,6900,35.384615,47.366898,8.528817,261,Zürich


### Reading municipality data

In [5]:
# Meaning of variables:
# bfs_number: official municipality id
# bfs_name: official municipality name
# pop: number of residents (=population)
# pop_dens: population density (pop per km2)
# frg_pct: percentage foreigners
# emp: numer of employees

df_municip = pd.read_excel('municipality_data.xlsx', 
                           sheet_name='data_for_import')
df_municip.head(5)

,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp,tax_ income,mean_taxable_income
0,1,Aeugst am Albis,1981,250.442478,14.184755,442.0,105013.570634,105013.570634
1,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,71059.805603,71059.805603
2,3,Bonstetten,5572,749.932705,16.564968,1014.0,88927.698145,88927.698145
3,4,Hausen am Albis,3751,275.808824,16.022394,1021.0,86300.455137,86300.455137
4,5,Hedingen,3778,578.560490,16.410799,1478.0,90811.205330,90811.205330


### Join municipality data to rental apartment data using .merge()

In [6]:
# Merge needs a key which must be identical in both data sets (here the key is 'bfs_number')
df3 = df2.merge(df_municip[['bfs_number', 
                            'pop', 
                            'pop_dens', 
                            'frg_pct', 
                            'emp',
                           'tax_ income',
                           'mean_taxable_income']], #from excel (task2)
                on="bfs_number")
df3.head(5)

,web-scraper-order,address_raw,datetime,rooms,area,luxurious,price,price_per_m2,lat,lon,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp,tax_ income,mean_taxable_income
0,1662023695-433,"Sunnenbergstrasse 15, 8633 Wolfhausen, ZH",2023-01-21 17:05:04,3.5,122,1,3180,26.065574,47.255714,8.804976,112,Bubikon,7344,632.558140,11.410675,3617.0,79385.904340,76289.408707
1,1662023720-634,"Blumenbergstrasse 7, 8633 Wolfhausen, ZH",2023-01-21 17:05:04,4.0,87,0,1690,19.425287,47.254879,8.793746,112,Bubikon,7344,632.558140,11.410675,3617.0,79385.904340,76289.408707
2,1662023745-834,"8608 Bubikon, ZH",2023-01-21 17:05:04,3.5,92,0,2350,25.543478,47.277386,8.800306,112,Bubikon,7344,632.558140,11.410675,3617.0,79385.904340,76289.408707
3,1662023701-503,"8608 Bubikon, ZH",2023-01-21 17:05:04,3.5,130,0,2500,19.230769,47.277386,8.800306,112,Bubikon,7344,632.558140,11.410675,3617.0,79385.904340,76289.408707
4,1662023745-820,"Lavaterstr. 63, 8002 Zürich, ZH",2023-01-21 17:05:04,2.5,78,0,3760,48.205128,47.361378,8.533339,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704,65617.333884


### Export data to file

In [7]:
### Export apartment data to file
df3.to_csv('apartments_data_enriched.csv',
            sep=',',
            encoding='utf-8')

### Sorting data

In [8]:
# Sorting data by 'price' and 'area' with highest price above (ascending=False)
df3.sort_values(by=['price', 'area'], 
                ascending=False).head()

,web-scraper-order,address_raw,datetime,rooms,area,luxurious,price,price_per_m2,lat,lon,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp,tax_ income,mean_taxable_income
76,1662023771-1030,"Parkring 57, 8002 Zürich, ZH",2023-01-21 17:05:04,5.0,187,1,8900,47.593583,47.366734,8.528435,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704,65617.333884
172,1662023689-385,"8053 Zürich, ZH",2023-01-21 17:05:04,4.5,171,1,7900,46.198830,47.358616,8.578999,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704,65617.333884
36,1662023733-742,"Giessereistrasse 12, 8005 Zürich, ZH",2023-01-21 17:05:04,2.5,282,0,7500,26.595745,47.390221,8.518072,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704,65617.333884
5,1662023739-771,"Parkring 59, 8002 Zürich, ZH",2023-01-21 17:05:04,5.5,195,1,6900,35.384615,47.366898,8.528817,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704,65617.333884
18,1662023786-1149,"Freudenbergstrasse 94, 8044 Zürich, ZH",2023-01-21 17:05:04,4.5,128,0,6630,51.796875,47.384132,8.556532,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704,65617.333884


### Aggregation of data

In [9]:
# Aggregation using .groupby()
df3[['rooms', 'price']].groupby(['rooms']).mean()

,price
rooms,
1.0,1501.477273
1.5,1904.409091
2.0,1562.805556
2.5,2273.563758
3.0,1825.421875
3.5,2339.687023
4.0,2261.100000
4.5,2650.764368
5.0,3257.230769


## Reshaping data

### Reshaping data using .stack() and .unstack()

In [10]:
df_sub = df3[['bfs_name', 'rooms', 'price', 'area']][:5]
print('Original shape')
print(df_sub, '\n')

df_sub_stacked = df_sub.stack()
print('Stacked')
print(df_sub_stacked, '\n')

# Using unstack
print('Unstacked (= back to original shape)')
print(df_sub_stacked.unstack())

Original shape
  bfs_name  rooms  price  area
0  Bubikon    3.5   3180   122
1  Bubikon    4.0   1690    87
2  Bubikon    3.5   2350    92
3  Bubikon    3.5   2500   130
4   Zürich    2.5   3760    78 

Stacked
0  bfs_name    Bubikon
   rooms           3.5
   price          3180
   area            122
1  bfs_name    Bubikon
   rooms           4.0
   price          1690
   area             87
2  bfs_name    Bubikon
   rooms           3.5
   price          2350
   area             92
3  bfs_name    Bubikon
   rooms           3.5
   price          2500
   area            130
4  bfs_name     Zürich
   rooms           2.5
   price          3760
   area             78
dtype: object 

Unstacked (= back to original shape)
  bfs_name rooms price area
0  Bubikon   3.5  3180  122
1  Bubikon   4.0  1690   87
2  Bubikon   3.5  2350   92
3  Bubikon   3.5  2500  130
4   Zürich   2.5  3760   78


### Reshaping data using .melt()

In [11]:
df_sub = df3[['rooms', 'price', 'area']][:5]
print('Original shape')
print(df_sub, '\n')

print('Reshaped using .melt()')
print(pd.melt(df, id_vars=['rooms'], value_vars=['price', 'area']))

Original shape
   rooms  price  area
0    3.5   3180   122
1    4.0   1690    87
2    3.5   2350    92
3    3.5   2500   130
4    2.5   3760    78 

Reshaped using .melt()
      rooms variable  value
0       3.5    price   3180
1       2.5    price   3760
2       5.5    price   2860
3       3.5    price   2165
4       5.5    price   6900
...     ...      ...    ...
1823    1.5     area     65
1824    3.5     area     70
1825    2.5     area     56
1826    3.5     area     70
1827    3.5     area     96

[1828 rows x 3 columns]


### Pivoting data using .pivot_table()

###### Exercise e: 
The relationship between the variable area and the price_per_m2 is that we divide price by area and get price_per_m2.
If we compare them, we see:
"The higher the price and the smaller the area, the higher the price per square metre."
In order to compare the average value (mean), we need to take into account the number of apartments.
Since mean-values are susceptible to outliers.

##### Exercise f: 
If we were to plot the data on a graph, we would get approximately a normal distribution whose maximum is 3.5 rooms with an 262 apartements.

In [12]:
# Using pivot_table to reshape the data and calculate means 

pd.pivot_table(df3[['rooms', 'price', 'area','price_per_m2']], #price_per_m2 was added (task 3)
               index=['rooms'],
               values=['price', 'area', 'price_per_m2'],
               aggfunc=(np.mean,'count' )) #'count' added as apartments-counter)

area             price              price_per_m2           
      count        mean count         mean        count       mean
rooms                                                             
1.0      44   43.113636    44  1501.477273           44  42.640017
1.5      44   61.909091    44  1904.409091           44  42.319048
2.0      36   55.194444    36  1562.805556           36  29.221596
2.5     149   68.704698   149  2273.563758          149  33.974370
3.0      64   68.781250    64  1825.421875           64  27.065527
3.5     262   87.240458   262  2339.687023          262  27.057849
4.0      50   87.360000    50  2261.100000           50  26.043648
4.5     174  110.764368   174  2650.764368          174  24.147628
5.0      13  124.769231    13  3257.230769           13  24.828969
5.5      29  140.068966    29  3013.310345           29  21.721217
6.0       1  153.000000     1  4240.000000            1  27.712418
6.5       1  194.000000     1  4760.000000            1  24.536082
9.0       1  300.000000     1  4000.000000            1  13.333333

In [13]:
# Using pivot_table to reshape the data and calculate means 

pd.pivot_table(df3[['rooms', 'price', 'area','price_per_m2']], #price_per_m2 was added (task 3)
               index=['rooms'],
               values=['price', 'area', 'price_per_m2'],
               aggfunc=('count')) #() & 'count' were added (apartments-counter)

,area,price,price_per_m2
rooms,,,
1.0,44,44,44
1.5,44,44,44
2.0,36,36,36
2.5,149,149,149
3.0,64,64,64
3.5,262,262,262
4.0,50,50,50
4.5,174,174,174
5.0,13,13,13


In [14]:
# sorted data -test
df3.sort_values(by=['price','area'], 
                ascending= False) [['rooms', 'price', 'area','price_per_m2']]

,rooms,price,area,price_per_m2
76,5.0,8900,187,47.593583
172,4.5,7900,171,46.198830
36,2.5,7500,282,26.595745
5,5.5,6900,195,35.384615
18,4.5,6630,128,51.796875
...,...,...,...,...
839,2.0,895,54,16.574074
379,1.0,850,34,25.000000
694,1.0,550,12,45.833333
157,4.5,24,167,0.143713


In [15]:
# Using pivot_table to reshape the data and calculate means 

pd.pivot_table(df3[['bfs_name','rooms', 'price', 'area','price_per_m2']], 
               index=['bfs_name'],
               values=['rooms','price', 'area', 'price_per_m2'],
               aggfunc=np.mean).sort_values(by=['price','area'], ascending= False) 


,area,price,price_per_m2,rooms
bfs_name,,,,
Küsnacht (ZH),124.00,4685.0,40.482402,3.333333
Kappel am Albis,300.00,4000.0,13.333333,9.000000
Oberengstringen,138.00,3725.0,26.992754,4.500000
Uitikon,143.00,3333.0,23.307692,4.500000
Thalwil,102.00,3075.0,31.640625,4.750000
...,...,...,...,...
Flurlingen,48.00,1490.0,31.041667,2.000000
Feuerthalen,82.75,1467.5,17.817097,4.125000
Niederweningen,63.00,1360.0,21.587302,2.500000
